# Workflow for the calibration and the analysis of a population-average model 

1. Parameter estimation using Maximum Likelihood Estimation
2. Calculation of Profile Likelihoods for each parameter
3. MCMC sampling
4. Sobol analysis

The pypesto ecosystem (https://pypesto.readthedocs.io/en/latest/index.html) is used to perform parameter estimation, profile likelihoods and MCMC sampling

In [1]:
import pypesto
import pypesto.optimize as optimize
import pypesto.profile as profile
import pypesto.visualize as visualize
import pypesto.sample as sample
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from SALib.sample import saltelli
from SALib.analyze import sobol
import seaborn as sns

## Load pre-processed data

continous mean and standart deviation of BGS2 data

In [2]:
FSH_moving = pd.read_csv('DataPoints/FSH_rollingData.txt', header=None)
E2_moving = pd.read_csv('DataPoints/E2_rollingData.txt', header=None)
LH_moving = pd.read_csv('DataPoints/LH_rollingData.txt', header=None)

## Define model, parameters, inital condition

In [3]:
#start value originate from a differential evolution test run
P = {"k_syn_FSH": 49.49418845,
    "k_cl_FSH": 0.10008877, 
    "k_syn_E2": 12.95594545,
    "k_cl_E2": 0.10001612,
    "T_FSH_E2": 2.86346153,
    "n_FSH_E2": 5.20396759,
    "k_syn_LH": 0.10001534,
    "k_cl_LH": 0.53593832,
    "T_E2_LH": 0.10587896,
    "n_E2_LH": 2.19039377, 
    "log_growth": 0.37592248, 
    "log_mid": 19.83817588}

y_data = [FSH_moving[1].values, E2_moving[1].values, LH_moving[1].values]
t_data  = [FSH_moving[0].values, E2_moving[0].values, LH_moving[0].values]
sigma_data = [FSH_moving[2].values, E2_moving[2].values, LH_moving[2].values]

t_eval = np.concatenate((FSH_moving[0].values, E2_moving[0].values, LH_moving[0].values))
t_eval = np.unique(t_eval)
y0 = [y_data[0][0], y_data[1][0], y_data[2][0]] 
t_init = 7.5
t_final = 11.5
t_span = [t_init, t_final]

def dy_dt(t, y):
    
    FSH, E2, LH = y
    
    dFSH = P['k_syn_FSH'] * (1/(1 + np.exp(-P['log_growth']*(t - P['log_mid'])))) - P['k_cl_FSH']*FSH
    dE2 = P['k_syn_E2'] * ((FSH/P['T_FSH_E2'])**P['n_FSH_E2'])/(1+(FSH/P['T_FSH_E2'])**P['n_FSH_E2']) - P['k_cl_E2']*E2
    dLH = P['k_syn_LH'] * (1/(1 + np.exp(-P['log_growth']*(t - P['log_mid'])))) * ((E2/P['T_E2_LH'])**P['n_E2_LH'])/(1+(E2/P['T_E2_LH'])**P['n_E2_LH']) - P['k_cl_LH']*LH

    dydt = [dFSH, dE2, dLH]
    return dydt

## Define likelihood function for MLE and MCMC

In [4]:
def Objective(sol_y, sol_t, y_data, t_data, sigma_data):

    H = np.zeros(1)
    
    for j in range(np.shape(y_data)[0]):
        for i in range(len(y_data[j])):
            t_index = np.where(sol_t == t_data[j][i])
            H[0] += (1/sigma_data[j][i]**2)*(y_data[j][i] - sol_y[j][t_index])**2
    
    return H

def CalcObjective(x):
    # x = [k_syn_FSH, k_cl_FSH, ...]
    # args = [y_data, t_data]
    P["k_syn_FSH"] = x[0]
    P["k_cl_FSH"] = x[1]
    P["k_syn_E2"] = x[2]
    P["k_cl_E2"] = x[3]
    P["T_FSH_E2"] = x[4]
    P["n_FSH_E2"] = x[5]
    P["k_syn_LH"] = x[6]
    P["k_cl_LH"] = x[7]
    P["T_E2_LH"] = x[8]
    P["n_E2_LH"] = x[9]
    P["log_growth"] = x[10]
    P["log_mid"] = x[11]

    sol = solve_ivp(dy_dt, t_span, y0, t_eval=t_eval)
    
    return float(Objective(sol.y, sol.t, y_data, t_data, sigma_data))

## MLE: multi-start optimisation

In [5]:
#set number of starts 
n_starts = 500

In [ ]:
custom_objective = pypesto.Objective(fun=CalcObjective, grad='3-point')

lb = np.array([0.001, 0.0000001, 0.001, 0.0000001, 0.001, 1.0, 0.001, 0.001, 0.001, 1.0, 0.001, 5.0])
ub = np.array([150.0, 150.0, 150.0, 150.0, 20.0, 5.0, 150.0, 150.0, 10.0, 5.0, 10.0, 25.0])

problem = pypesto.Problem(objective=custom_objective, lb=lb, ub=ub)
optimizer = optimize.ScipyOptimizer(method='L-BFGS-B')

history_options = pypesto.HistoryOptions(trace_record=True)

result = optimize.minimize(problem=problem,optimizer=optimizer,n_starts=n_starts,
                              history_options=history_options,filename=None,)

  0%|          | 0/500 [00:00<?, ?it/s]Executing task 0.
Final fval=90.9066, time=50.0834s, n_fval=2197.
  0%|          | 1/500 [00:50<6:56:32, 50.09s/it]Executing task 1.
Final fval=226.2890, time=39.6199s, n_fval=1170.
  0%|          | 2/500 [01:29<6:04:37, 43.93s/it]Executing task 2.
Final fval=44.0647, time=76.4362s, n_fval=4511.
  1%|          | 3/500 [02:46<8:06:51, 58.78s/it]Executing task 3.
Final fval=861.1349, time=24.6332s, n_fval=728.
  1%|          | 4/500 [03:10<6:14:29, 45.30s/it]Executing task 4.
Final fval=8.2840, time=114.6796s, n_fval=9581.
  1%|          | 5/500 [05:05<9:40:08, 70.32s/it]Executing task 5.
Final fval=253.4396, time=17.4736s, n_fval=481.
  1%|          | 6/500 [05:22<7:11:02, 52.35s/it]Executing task 6.
Parameters obtained from history and optimizer do not match: [3.23742265e+01 1.00000000e-07 1.13054902e+02 6.08534978e+01
 1.03001155e-03 3.25213254e+00 3.09812623e+01 1.00000000e-03
 8.72271581e+00 3.23135700e+00 3.46376463e+00 2.11679379e+01], [3.237

### convert optimisation results in data frame and plot

In [ ]:
df_optiresults = result.optimize_result.as_dataframe(["x","fval","time", "grad", "x0", "exitflag"])

fig, ax = plt.subplots(figsize=(15, 5))

ax.plot(df_optiresults['fval'], 'x', color='grey', alpha=0.5)
plt.yscale("log")
ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='out')
ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='out')
ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='out')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set(xlabel='run index (sorted by likelihood)', ylabel='- log$\mathcal{L}(\hat \\theta)$')

P['k_syn_FSH'] = df_optiresults.x[0][0]
P['k_cl_FSH'] = df_optiresults.x[0][1]
P["k_syn_E2"] = df_optiresults.x[0][2]
P["k_cl_E2"] = df_optiresults.x[0][3]
P["T_FSH_E2"] = df_optiresults.x[0][4]
P["n_FSH_E2"] = df_optiresults.x[0][5]
P["k_syn_LH"] = df_optiresults.x[0][6]
P["k_cl_LH"] = df_optiresults.x[0][7]
P["T_E2_LH"] = df_optiresults.x[0][8]
P["n_E2_LH"] = df_optiresults.x[0][9]
P["log_growth"] = df_optiresults.x[0][10]
P["log_mid"] = df_optiresults.x[0][11]
sol = solve_ivp(dy_dt, t_span, y0, t_eval=t_eval)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
ax[0].scatter(t_data[0], y_data[0], color='#e0ca3cff', alpha=0.1)
#ax[0].scatter(FSH_all[0], FSH_all[1], color='grey', alpha=0.3, marker='x')
ax[0].errorbar(t_data[0][::10], y_data[0][::10], yerr=sigma_data[0][::10], color='#e0ca3cff', fmt="o")
ax[0].plot(sol.t, sol.y[0], color='#5d80c0ff', lw=3)
ax[0].set_xlim(7.5, 11.25)
ax[0].set_ylim(0.0, 8.5)

ax[1].scatter(t_data[1], y_data[1], color='#e0ca3cff', alpha=0.1)
#ax[1].scatter(E2_all[0], E2_all[1], color='grey', alpha=0.3, marker='x')
ax[1].errorbar(t_data[1][::10], y_data[1][::10], yerr=sigma_data[1][::10], color='#e0ca3cff', fmt="o")
ax[1].plot(sol.t, sol.y[1], color='#5d80c0ff', lw=3)
ax[1].set_xlim(7.5, 11.25)
ax[1].set_ylim(0.0, 8.5)

ax[2].scatter(t_data[2], y_data[2], color='#e0ca3cff', alpha=0.1)
#ax[2].scatter(LH_all[0], LH_all[1], color='grey', alpha=0.3, marker='x')
ax[2].errorbar(t_data[2][::10], y_data[2][::10], yerr=sigma_data[2][::10], color='#e0ca3cff', fmt="o")
ax[2].plot(sol.t, sol.y[2], color='#5d80c0ff', lw=3)
ax[2].set_xlim(7.5, 11.25)
ax[2].set_ylim(0.0, 3.5)

## Calculate and plot Profile Likelihoods

In [ ]:
result = profile.parameter_profile(problem=problem, result=result, optimizer=optimizer, filename=None)

In [ ]:
ax = visualize.profiles(result,)

## MCMC sampling 
-> decreasing the number of starts (n_start) will decrease the run time, but also effect the optimisation outcome

In [ ]:
num_samples =100000

P = {"k_syn_FSH": df_optiresults['x'][0][0],
     "k_cl_FSH": df_optiresults['x'][0][1],
     "k_syn_E2": df_optiresults['x'][0][2],
     "k_cl_E2": df_optiresults['x'][0][3],
     "T_FSH_E2": df_optiresults['x'][0][4],
     "n_FSH_E2": df_optiresults['x'][0][5],
     "k_syn_LH": df_optiresults['x'][0][6],
     "k_cl_LH": df_optiresults['x'][0][7],
     "T_E2_LH": df_optiresults['x'][0][8],
     "n_E2_LH": df_optiresults['x'][0][9],
     "log_growth": df_optiresults['x'][0][10], 
     "log_mid": df_optiresults['x'][0][11]}

lb = np.array([0.000000001, 0.000000001, 0.000000001, 0.000000001, 0.00000001, 0.00000001, 0.000000001, 0.000000001
              , 0.000000001, 0.000000001, 0.000000001, 0.000000001])
ub = np.array([150.0, 10.0, 150.0, 10.0, 50.0, 10.0, 150.0, 10.0, 50.0, 10.0, 10.0, 50.0])

params = np.array(list(P.values()))

sampler = sample.AdaptiveParallelTemperingSampler(internal_sampler=sample.AdaptiveMetropolisSampler(), n_chains=3)
result = sample.sample(problem, n_samples=num_samples, sampler=sampler, x0=params, filename=None)

ax = visualize.sampling_parameter_traces(result, use_problem_bounds=False, size=(25, 15))

## Sobol Analysis 
create problem for Sobol analysis (https://salib.readthedocs.io/en/latest/getting-started.html)

In [ ]:
def dy_dt(t, y, k_syn_FSH, k_cl_FSH, k_syn_E2, k_cl_E2, T_FSH_E2, n_FSH_E2, 
          k_syn_LH, k_cl_LH, T_E2_LH, n_E2_LH, sig_growth, sig_midpoint):

    FSH, E2, LH = y
    
    dFSH = k_syn_FSH * (1/(1+np.exp(-sig_growth*(t-sig_midpoint)))) - k_cl_FSH*FSH
    dE2 = k_syn_E2 * ((FSH/T_FSH_E2)**n_FSH_E2)/(1+(FSH/T_FSH_E2)**n_FSH_E2) - k_cl_E2*E2
    dLH = k_syn_LH * (1/(1+np.exp(-sig_growth*(t-sig_midpoint)))) * ((E2/T_E2_LH)**n_E2_LH)/(1+(E2/T_E2_LH)**n_E2_LH) - k_cl_LH*LH

    dydt = [dFSH, dE2, dLH]
    return dydt

k_syn_FSH = df_optiresults['x'][0][0]
k_cl_FSH = df_optiresults['x'][0][1]
k_syn_E2 = df_optiresults['x'][0][2]
k_cl_E2 = df_optiresults['x'][0][3]
T_FSH_E2 = df_optiresults['x'][0][4]
n_FSH_E2 = df_optiresults['x'][0][5]
k_syn_LH = df_optiresults['x'][0][6]
k_cl_LH = df_optiresults['x'][0][7]
T_E2_LH = df_optiresults['x'][0][8]
n_E2_LH = df_optiresults['x'][0][9]
sig_growth = df_optiresults['x'][0][10]
sig_midpoint = df_optiresults['x'][0][11]

problem = {
  'num_vars': 15, 
  'names': ['k_syn_FSH', 'k_cl_FSH', 'k_syn_E2', 'k_cl_E2', 'T_FSH_E2', 'n_FSH_E2',
            'k_syn_LH', 'k_cl_LH', 'T_LH_E2', 'n_E2_LH', 'sig_growth','sig_midpoint', 
            'x1_0', 'x2_0', 'x3_0'],
  'bounds':  np.column_stack((np.array([k_syn_FSH, k_cl_FSH, k_syn_E2, k_cl_E2, T_FSH_E2, 
                                        n_FSH_E2,k_syn_LH, k_cl_LH, T_E2_LH, n_E2_LH, sig_growth,sig_midpoint,
                                        y0[0], y0[1],y0[2]])*0.8,
                              np.array([k_syn_FSH, k_cl_FSH, k_syn_E2, k_cl_E2, T_FSH_E2, 
                                        n_FSH_E2, k_syn_LH, k_cl_LH, T_E2_LH, n_E2_LH, sig_growth,sig_midpoint,
                                        y0[0], y0[1], y0[2]])*1.2))
}

# Generate samples
##N should be a power of 2 value eg 512
N=11
vals = saltelli.sample(problem, 2**N, calc_second_order=False)

# initializing matrix to store output
Y = np.zeros((len(vals),3,len(t_eval)))

# Run model (example)
# numerically soves the ODE
# output is X1, X2, and X3 at the end time step
# could save output for all time steps if desired, but requires more memory

for i in range(len(vals)):
    sol = solve_ivp(dy_dt, t_span, [vals[i][12], vals[i][13], vals[i][14]], t_eval=t_eval, 
                    args=(vals[i][0], vals[i][1], vals[i][2], vals[i][3], vals[i][4], vals[i][5], 
                          vals[i][6], vals[i][7], vals[i][8], vals[i][9], vals[i][10], vals[i][11])).y
    for k in range(len(t_eval)): 
        Y[i,0,k] = sol[0][k]
        Y[i,1,k] = sol[1][k]
        Y[i,2,k] = sol[2][k]

### Calculate Sobol index at each time point for each parameter and species

In [ ]:
FSH_total = np.zeros((15,len(t_eval)))
FSH_first = np.zeros((15,len(t_eval)))
E2_total = np.zeros((15,len(t_eval)))
E2_first = np.zeros((15,len(t_eval)))
LH_total = np.zeros((15,len(t_eval)))
LH_first = np.zeros((15,len(t_eval)))

for k in range(len(t_eval)): 
    Si_FSH = sobol.analyze(problem, Y[:,0, k], calc_second_order=False, print_to_console=False)
    Si_E2 = sobol.analyze(problem, Y[:,1, k], calc_second_order=False, print_to_console=False)
    Si_LH = sobol.analyze(problem, Y[:,2, k], calc_second_order=False, print_to_console=False)
    total, first = Si_FSH.to_df()
    FSH_total[:,k] = total['ST']
    FSH_first[:,k] = first['S1']
    total, first = Si_E2.to_df()
    E2_total[:,k] = total['ST']
    E2_first[:,k] = first['S1']
    total, first = Si_LH.to_df()
    LH_total[:,k] = total['ST']
    LH_first[:,k] = first['S1']

### Plot results as heatmaps

In [ ]:
param_keys = ['k_syn_FSH', 'k_cl_FSH', 'k_syn_E2', 'k_cl_E2', '$T_{FSH}^{E2}$', '$n_{FSH}^{E2}$',
            'k_syn_LH', 'k_cl_LH', 'T_LH_E2', 'n_E2_LH', '$f_s$','$f_m$', 
            '$FSH_0$', '$E2_0$', '$LH_0$']

In [ ]:
ax = sns.heatmap(FSH_first[:, :2], yticklabels=param_keys)
plt.show()
ax = sns.heatmap(FSH_total[:, :2], yticklabels=param_keys)
plt.show()

In [ ]:
ax = sns.heatmap(E2_first[:, :2], yticklabels=param_keys)
plt.show()
ax = sns.heatmap(E2_total[:, :2], yticklabels=param_keys)
plt.show()

In [ ]:
ax = sns.heatmap(LH_first[:, :2], yticklabels=param_keys)
plt.show()
ax = sns.heatmap(LH_total[:, :2], yticklabels=param_keys)
plt.show()